# Key Concepts


Welcome to `braintrace`!

BrainTrace is a Python library designed for implementing online learning in neural network models with dynamics. Online learning represents a learning paradigm that enables continuous parameter updates as models receive new data streams. This approach proves particularly valuable in real-world applications, including robotic control systems, agent decision-making processes, and large-scale data stream processing.


In this section, I will introduce some of the key concepts that are fundamental to understanding and using online learning methods defined in ``braintrace`` . These concepts include:

- Concepts related to build high-Level Neural Networks.
- Concepts related to customize neural network module: ``ETraceVar`` for hidden states, ``ETraceParam`` for weight parameters, and ``ETraceOp`` for input-to-hidden transition.
- Concepts for online learning algorithms ``ETraceAlgorithm``.

``braintrace`` is seamlessly integrated in the [brain dynamics programming ecosystem](https://brainmodeling.readthedocs.io/) centred on ``brainstate``. We strongly recommend that you first familiarise yourself with [basic usage of ``brainstate``](https://brainstate.readthedocs.io/), as this will help you better understand how ``braintrace`` works.


In [1]:
import brainstate 
import brainunit as u
import braintrace
import braintools
import brainpy

## 1. Dynamical Models Supported in ``braintrace``

``BrainTrace`` does not support online learning for arbitrary dynamical models. The dynamical models currently supported by ``BrainTrace`` exhibit a specific architectural constraint, as illustrated in the figure below, wherein the "dynamics" and "interactions between dynamics" are strictly segregated. Models adhering to this architecture can be decomposed into two primary components:

- **Dynamics**: This component characterizes the intrinsic dynamics of neurons, encompassing models such as the Leaky Integrate-and-Fire (LIF) neuron model, the FitzHugh-Nagumo model, and Long Short-Term Memory (LSTM) networks. The update of dynamics (hidden states) is implemented through strictly element-wise operations, although the model may incorporate multiple hidden states.

- **Interaction between Dynamics**: This component defines the interactions between neurons, implemented through weight matrices or connectivity matrices. The interactions between model dynamics can be realized through standard matrix multiplication, convolutional operations, or sparse operations.

![](../_static/model-dynamics-supported.png)




To elucidate the class of dynamical models supported by BrainTrace, let us examine a fundamental Leaky Integrate-and-Fire (LIF) neural network model. The dynamics of this network are governed by the following differential equations:

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + s_i \\
\tau_s \frac{ds_i}{dt} &= -s_i + \sum_{j} w_{ij} \delta(t - t_j)
\end{aligned}
$$

Here, $v_i$ represents the membrane potential of the neuron. When this potential exceeds a threshold value $v_{th}$, the neuron generates an action potential and its membrane potential is reset to $v_{\text{reset}}$, as described by:

$$
\begin{aligned}
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}}
\end{aligned}
$$

Additionally, $s_i$ denotes the postsynaptic current, $I_{\text{ext}}$ represents the external input current, $w_{ij}$ is the synaptic weight from neuron $i$ to neuron $j$, and $\delta(t - t_j)$ is the Dirac delta function indicating the reception of a synaptic event at time $t_j$. The time constants $\tau$ and $\tau_s$ characterize the temporal evolution of the membrane potential and postsynaptic current, respectively.

Through numerical integration, we discretize the above differential equations and express them in vector form, yielding the following update rules:

$$
\begin{aligned}
\mathbf{v}_i^{t+1} &= \mathbf{v}_i^{t} + \frac{\Delta t}{\tau} (-\mathbf{v}_i^{t} + \mathbf{I}_{\text{ext}} + \mathbf{s}^t) \\
\mathbf{s}_i^{t+1} &= \mathbf{s}_i^{t} + \frac{\Delta t}{\tau_s} (-\mathbf{s}_i^{t} + \underbrace{  W \mathbf{z}^t  } _ {\text{neuronal interaction}} )
\end{aligned}
$$

Notably, the dynamics of the LIF neurons are updated through element-wise operations, while the interaction component is implemented via matrix multiplication. All dynamical models supported by BrainTrace can be decomposed into similar `dynamics` and `interaction` components. It is particularly worth noting that this architecture encompasses the majority of recurrent neural network models, thus enabling BrainTrace to support online learning for a wide range of recurrent neural network architectures.

## 2. `braintrace.nn`: Constructing Neural Networks with Online Learning Support

The construction of neural network models supporting online learning in BrainTrace follows identical conventions as those employed in `brainstate`. For comprehensive tutorials, please refer to the documentation on [Building Artificial Neural Networks](https://brainstate.readthedocs.io/tutorials/examples/11_artificial_neural_networks.html) and [Simulating Spiking Neural Networks](https://brainstate.readthedocs.io/tutorials/examples/13_snn_simulation.html).

The sole distinction lies in the requirement to utilize components from the [`braintrace.nn` module](../apis/nn.rst) for model construction. These components represent extensions of `brainstate.nn` module, specifically engineered to provide modular units with online learning capabilities.

Below, we present a basic implementation demonstrating the construction of a Leaky Integrate-and-Fire (LIF) neural network using the `braintrace.nn` module.

In [2]:
class LIF_Delta_Net(brainstate.nn.Module):
    def __init__(
        self,
        n_in,
        n_rec,
        tau_mem=5. * u.ms,
        V_th=1. * u.mV,
        spk_fun=braintools.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        rec_scale: float = 1.,
        ff_scale: float = 1.,
    ):
        super().__init__()

        # Using the LIF model in braintrace.nn
        self.neu = brainpy.state.LIF(n_rec, tau=tau_mem, spk_fun=spk_fun, spk_reset=spk_reset, V_th=V_th)

        # Constructing input and recurrent connection weights
        rec_init = braintools.init.KaimingNormal(rec_scale, unit=u.mV)
        ff_init = braintools.init.KaimingNormal(ff_scale, unit=u.mV)
        w_init = u.math.concatenate([ff_init([n_in, n_rec]), rec_init([n_rec, n_rec])], axis=0)

        # Using delta synaptic projections to construct input and recurrent connections
        self.syn = brainpy.state.DeltaProj(
            # Using the Linear model in braintrace.nn
            comm=braintrace.nn.Linear(n_in + n_rec, n_rec, w_init=w_init, b_init=braintools.init.ZeroInit(unit=u.mV)),
            post=self.neu
        )

    def update(self, spk):
        inp = u.math.concatenate([spk, self.neu.get_spike()], axis=-1)
        self.syn(inp)
        self.neu()
        return self.neu.get_spike()

In this exemplar implementation, we define a `LIF_Delta_Net` class that inherits from `brainstate.nn.Module`. The architecture incorporates two primary components: a Leaky Integrate-and-Fire (LIF) neuron model implemented as `self.neu`, and a `DeltaProj` module designated as `self.syn` which manages both input and recurrent connectivity.

Subsequently, we shall proceed to construct a three-layer Gated Recurrent Unit (GRU) neural network model:

In [3]:
class GRU_Net(brainstate.nn.Module):
    def __init__(
        self,
        n_in: int,
        n_rec: int,
        n_out: int,
        n_layer: int,
    ):
        super().__init__()

        # Building the GRU Layer
        self.layers = []
        for i in range(n_layer - 1):
            # Using the GRUCell model within braintrace.nn
            self.layers.append(braintrace.nn.GRUCell(n_in, n_rec))
            n_in = n_rec
        self.layers.append(braintrace.nn.GRUCell(n_in, n_out))

    def update(self, x):
        # Updating the GRU Layer
        for layer in self.layers:
            x = layer(x)
        return x

As demonstrated, the process of constructing neural network models using the [`braintrace.nn` module](../apis/nn.rst) maintains complete procedural equivalence with the construction methodology employed in the [`brainstate.nn` module](https://brainstate.readthedocs.io/apis/nn.html). This architectural parallelism enables direct utilization of existing `brainstate` tutorials for developing neural network models with online learning capabilities.

## 3. `ETraceState`, `ETraceParam`, and `ETraceOp`: Customizing Network Modules

While the `braintrace.nn` module provides fundamental network components, it does not encompass all possible network dynamics. Consequently, BrainTrace implements a mechanism for customizing module development through three primary classes: `ETraceState`, `ETraceParam`, and `ETraceOp`.

**Core Components**

- **`brainstate.HiddenState`**: Represents the hidden states $\mathbf{h}$ within modules, defining dynamical states such as membrane potentials in LIF neurons or postsynaptic conductances in exponential synaptic models.

- **`braintrace.ETraceOp`**: Describe network connections, or how input data is used to compute postsynaptic currents based on model parameters, such as linear matrix multiplication, sparse matrix multiplication, and convolution operations.

- **`braintrace.ETraceParam`**: Corresponds to model parameters $\theta$ within modules, encompassing elements such as weight matrices for linear matrix multiplication and adaptive time constants in LIF neurons. All parameters requiring gradient updates during training must be defined within `ETraceParam`.


**Foundational Framework**

These three components—`ETraceState`, `ETraceParam`, and `ETraceOp`—constitute the fundamental conceptual framework underlying neural network models with online learning capabilities in BrainTrace.

In the following sections, we will present a series of illustrative examples demonstrating the practical implementation of custom network modules using `ETraceState`, `ETraceParam`, and `ETraceOp`.

### 3.1 `ETraceState`: Model State

Let us first examine a fundamental Leaky Integrate-and-Fire (LIF) neuron model, whose dynamics are characterized by the following differential equations:

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + v_\text{rest} \\
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}} \quad \text{if} \quad z_i > 0
\end{aligned}
$$

Here, $v_i$ represents the membrane potential of the neuron. When this potential exceeds a threshold value $v_{th}$, the neuron generates an action potential, and its membrane potential is reset to $v_{\text{reset}}$. The Heaviside function $\mathcal{H}$ characterizes the action potential generation, while $I_{\text{ext}}$ denotes the external input current. The temporal evolution of the membrane potential is governed by the time constant $\tau$, and $v_\text{rest}$ represents the resting membrane potential.

In [4]:
import jax
from typing import Callable


class LIF(brainpy.state.Neuron):
    """
    Leaky integrate-and-fire neuron model.
    """

    def __init__(
        self,
        size: brainstate.typing.Size,
        R: brainstate.typing.ArrayLike = 1. * u.ohm,
        tau: brainstate.typing.ArrayLike = 5. * u.ms,
        V_th: brainstate.typing.ArrayLike = 1. * u.mV,
        V_reset: brainstate.typing.ArrayLike = 0. * u.mV,
        V_rest: brainstate.typing.ArrayLike = 0. * u.mV,
        V_initializer: Callable = braintools.init.Constant(0. * u.mV),
        spk_fun: Callable = braintools.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        name: str = None,
    ):
        super().__init__(size, name=name, spk_fun=spk_fun, spk_reset=spk_reset)

        # parameters
        self.R = braintools.init.param(R, self.varshape)
        self.tau = braintools.init.param(tau, self.varshape)
        self.V_th = braintools.init.param(V_th, self.varshape)
        self.V_rest = braintools.init.param(V_rest, self.varshape)
        self.V_reset = braintools.init.param(V_reset, self.varshape)
        self.V_initializer = V_initializer

    def init_state(self, batch_size: int = None, **kwargs):
        # Here is the most critical step, we define an ETraceState class 
        # that describes the kinetic state of membrane potentials
        self.V = brainstate.HiddenState(
            braintools.init.param(self.V_initializer, self.varshape, batch_size))

    def reset_state(self, batch_size: int = None, **kwargs):
        self.V.value = braintools.init.param(self.V_initializer, self.varshape, batch_size)

    def get_spike(self, V=None):
        V = self.V.value if V is None else V
        v_scaled = (V - self.V_th) / (self.V_th - self.V_reset)
        return self.spk_fun(v_scaled)

    def update(self, x=0. * u.mA):
        last_v = self.V.value
        lst_spk = self.get_spike(last_v)
        V_th = self.V_th if self.spk_reset == 'soft' else jax.lax.stop_gradient(last_v)
        V = last_v - (V_th - self.V_reset) * lst_spk
        # membrane potential
        dv = lambda v: (-v + self.V_rest + self.R * self.sum_current_inputs(x, v)) / self.tau
        V = brainstate.nn.exp_euler_step(dv, V)
        V = self.sum_delta_inputs(V)
        self.V.value = V
        return self.get_spike(V)

In the code above, we implement the `LIF` model through inheritance from `brainpy.state.Neuron`. The class incorporates an `ETraceState` class variable `self.V` that characterizes the dynamical state of the membrane potential. The `init_state` method establishes the initial conditions for the membrane potential dynamics, while the `update` method implements the temporal evolution of these dynamics.

This implementation maintains substantial structural similarity with the `LIF` class definition in `brainstate`, with one crucial distinction: whereas `brainstate` employs `brainstate.HiddenState` to represent the membrane potential dynamics, `braintrace` utilizes `brainstate.ETraceState` to explicitly designate this dynamical state for online learning applications.

Therefore, we say that `brainstate.HiddenState` can be conceptualized as the counterpart to `brainstate.HiddenState`, specifically designed for defining model states that require eligibility trace updates. Should model states be defined using `brainstate.HiddenState` rather than `brainstate.HiddenState`, the online learning compiler in `braintrace` will fail to recognize these states. This oversight results in the compiled online learning rules being ineffective for the affected states, potentially leading to erroneous or omitted gradient updates in the model.

But we should still be aware that there are obvious differences between `ETraceState` and `HiddenState`:
- `brainstate.HiddenState`: Explicitly marks states for eligibility trace computation
- `brainstate.HiddenState`: Standard state representation within ``branstate`` without online learning capabilities

### 3.2 `ETraceOp`: Model Connectivity

`ETraceOp` is a conceptual abstraction for describing model connectivity or how different dynamical components interact. It defines how model inputs are transformed into outputs based on specific parameters.


The standard usage format is:

```python
y = op(x, param)
```

Here, `x` represents the input data, `param` denotes the model parameters, and `y` is the resulting output. The core functionality of `ETraceOp` lies in applying a specific connection rule to combine the input data and parameters in order to compute the output.



BrainTrace provides several built-in operators for common modeling operations, including linear matrix multiplication, sparse matrix multiplication, and convolution. These include:

* [`braintrace.MatMulOp`](../apis/generated/braintrace.MatMulOp.rst): Standard matrix multiplication.
* [`braintrace.ConvOp`](../apis/generated/braintrace.ConvOp.rst): Standard convolution operation.
* [`braintrace.SpMatMulOp`](../apis/generated/braintrace.SpMatMulOp.rst): Sparse matrix-vector multiplication.
* [`braintrace.LoraOp`](../apis/generated/braintrace.LoraOp.rst): Low-Rank Adaptation (LoRA) operation.
* [`braintrace.ElemWiseOp`](../apis/generated/braintrace.ElemWiseOp.rst): Element-wise operations.


### 3.3 `ETraceParam` : Model Parameter

`ETraceParam` in BrainTrace is used to define trainable model parameters. It takes the following form:

```python
param = braintrace.ETraceParam(parameters, op)
```

Here, `parameters` refers to the model parameters, and `op` is an instantiated `ETraceOp`. The typical usage pattern is:

```python
y = param.execute(x)
```

Below, we use linear matrix multiplication as an example to demonstrate how to define a weight matrix and bias vector for a linear layer using `ETraceParam`.

In [5]:
def generate_weight(
    n_in, n_out, init: Callable = braintools.init.KaimingNormal()
) -> braintrace.ETraceParam:
    weight = init([n_in, n_out])
    bias = braintools.init.ZeroInit()([n_out])
    
    # Here is the most crucial step, we define an ETraceParam class to describe the weight matrix and bias vector
    return braintrace.ETraceParam(
        {'weight': weight, 'bias': bias},  # model parameters
        braintrace.MatMulOp()  # operation
    )

In the code above, we define a `generate_weight` function that produces weight matrices and bias vectors. This function returns an `ETraceParam` object that encapsulates these parameters.

`braintrace.ETraceParam` serves as the counterpart to `brainstate.ParamState`, specifically designed for model parameters requiring eligibility trace updates. When model parameters $\theta$ are defined using `braintrace.ETraceParam`, the online learning compiler in `braintrace` implements temporally-dependent gradient updates according to the following formula:

$$
\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \sum_{k=1}^t \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^k},
$$

where $\boldsymbol{\theta}^k$ represents the weight $\boldsymbol{\theta}$ utilized at time step $k$.

Conversely, when model parameters $\theta$ are defined using `brainstate.ParamState`, the online learning compiler computes only the instantaneous gradient of the loss function with respect to the weights:

$$
\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^t}.
$$

This implementation distinction signifies that in `braintrace`'s online learning framework, parameters defined as `brainstate.ParamState` are treated as those not requiring eligibility trace updates, thereby forfeiting the ability to compute gradients with temporal dependencies. This architectural design enhances the flexibility of parameter update patterns, thereby increasing the customizability of gradient computation mechanisms.


Besides, `ETraceParam` encapsulates both model parameters and their associated operations, providing a comprehensive framework for describing parameter-operation combinations. During instantiation, it requires both a parameter object and an operation function as inputs.

Using the `ETraceParam` framework, we can reformulate the matrix multiplication operator discussed above as follows:

In [6]:
class Linear(brainstate.nn.Module):
    """
    Linear layer.
    """
    def __init__(
        self,
        in_size: brainstate.typing.Size,
        out_size: brainstate.typing.Size,
        w_init: Callable = braintools.init.KaimingNormal(),
    ):
        super().__init__()

        # input and output shape
        self.in_size = in_size
        self.out_size = out_size

        # weights
        weight = braintools.init.param(w_init, [self.in_size[-1], self.out_size[-1]], allow_none=False)
        
        # operation
        op = braintrace.MatMulOp()
        
        # Here is the most crucial step, we define an ETraceParam class to describe the weight matrix and operations
        self.weight_op = braintrace.ETraceParam(weight, op)

    def update(self, x):
        # Operation of ETraceParam
        return self.weight_op.execute(x)

As demonstrated in the code above, `ETraceParam` represents an integrated construct that unifies model parameters and their associated operations. This unified framework requires both a parameter object and an operation function for instantiation. The `execute` method of `ETraceParam` implements the operational transformation, converting input data into output data according to the specified parameters and operations.

Through this implementation, we have successfully defined a fundamental `Linear` layer module, encompassing a weight matrix and its corresponding matrix multiplication operation. This module serves as a building block for constructing neural network models with online learning capabilities.

## 4. `ETraceAlgorithm`: Online Learning Algorithms

`ETraceAlgorithm` represents another fundamental concept in the BrainTrace framework, defining both the eligibility trace update process during model state evolution and the gradient update rules for model parameters. Implemented as an abstract class, `ETraceAlgorithm` serves as a specialized framework for describing various forms of online learning algorithms within BrainTrace.

The algorithmic support provided by `braintrace.ETraceAlgorithm` is founded upon the three fundamental concepts previously introduced: `ETraceState`, `ETraceParam`, and `ETraceOp`. 

`braintrace.ETraceAlgorithm` implements a flexible online learning compiler that enables online learning capabilities for any neural network model constructed using these three foundational concepts.

Specifically, BrainTrace currently supports the following online learning algorithms:

1. [`braintrace.IODimVjpAlgorithm`](../apis/generated/braintrace.IODimVjpAlgorithm.rst) or [`braintrace.ES_D_RTRL`](../apis/generated/braintrace.ES_D_RTRL.rst)
    - Implements the ES-D-RTRL algorithm for online learning
    - Achieves $O(N)$ memory and computational complexity for online gradient computation
    - Optimized for large-scale spiking neural network models
    - Detailed algorithm specifications are available in [our paper](https://doi.org/10.1101/2024.09.24.614728)

2. [`braintrace.ParamDimVjpAlgorithm`](../apis/generated/braintrace.ParamDimVjpAlgorithm.rst) or [`braintrace.D_RTRL`](../apis/generated/braintrace.D_RTRL.rst)
    - Utilizes the D-RTRL algorithm for online learning
    - Features $O(N^2)$ memory and computational complexity for online gradient computation
    - Applicable to both recurrent neural networks and spiking neural network models
    - Complete algorithmic details are documented in [our paper](https://doi.org/10.1101/2024.09.24.614728)

3. [`braintrace.HybridDimVjpAlgorithm`](../apis/generated/braintrace.HybridDimVjpAlgorithm.rst)
    - Implements selective application of ES-D-RTRL or D-RTRL algorithms for parameter updates
    - Preferentially employs D-RTRL for convolutional layers and highly sparse connections
    - Optimizes memory and computational complexity of parameter updates through adaptive algorithm selection

The framework is designed for extensibility, with ongoing development to support additional online learning algorithms for diverse application scenarios.

In the following demonstration, we will illustrate the process of constructing a neural network model with online learning capabilities using `braintrace.ETraceAlgorithm`. This example will serve to exemplify the practical implementation of the concepts discussed above.

In [7]:
with brainstate.environ.context(dt=0.1 * u.ms):

    # Define a simple recurrent neural network composed of LIF neurons
    model = LIF_Delta_Net(10, 10)
    brainstate.nn.init_all_states(model)
    
    # The model is fed into an online learning algorithm with a view to online learning
    model = braintrace.IODimVjpAlgorithm(model, decay_or_rank=0.99)
    
    # Compile the model's eligibility trace based on one input data. 
    # Thereafter, the model is called to update not only the state 
    # of the model, but also the model's eligibility trace
    example_input = brainstate.random.random(10) < 0.1
    model.compile_graph(example_input)

In essence, the user-defined neural network model solely specifies how the model states $\mathbf{h}$ evolve forward in time as a function of inputs. The compiled `ETraceAlgorithm`, in contrast, defines the update dynamics of the model's eligibility traces $\mathbf{\epsilon}$ in relation to state updates. Consequently, subsequent model invocations result in concurrent updates of both model states and their corresponding eligibility traces.

In [8]:
with brainstate.environ.context(dt=0.1 * u.ms):
    
    out = model(example_input)

# The eligibility trace of the model's pre-synaptic neural 
# activity trace can be obtained by calling "model.etrace_xs"
brainstate.util.PrettyMapping(model.etrace_xs)

{
  2152391028160: EligibilityTrace(
    value=ShapedArray(float32[20])
  )
}

In [9]:
# The eligibility trace of the model's post-synaptic neural 
# activity trace can be obtained by calling "model.etrace_dfs"
brainstate.util.PrettyMapping(model.etrace_dfs)

{
  (2152391028288, 'hidden_group_0'): EligibilityTrace(
    value=ShapedArray(float32[10,1])
  )
}

## 5. Conclusion

`BrainTrace` provides a comprehensive and elegant framework for online learning, with its core concepts organized into the following hierarchical layers:

1. Infrastructure Layer
    - Supports specific dynamical model architectures with strict separation between "dynamics" and "interactions"
    - Built upon the `BrainState` ecosystem, maintaining full compatibility with its programming paradigm
    - Provides ready-to-use neural network components through the `braintrace.nn` module

2. Core Concepts Layer
    - `ETraceState`: Designates hidden states requiring eligibility trace updates
    - `ETraceOp`: Defines specific operations for dynamical interactions
    - `ETraceParam`: Provides unified interface for parameter-operation combinations

3. Algorithm Implementation Layer
    - `DiagIODimAlgorithm`: Implements ES-D-RTRL algorithm with $O(N)$ complexity
    - `DiagParamDimAlgorithm`: Implements D-RTRL algorithm with $O(N^2)$ complexity
    - `DiagHybridDimAlgorithm`: Adaptive hybrid approach, selecting between $O(N)$ and $O(N^2)$ complexity algorithms based on network architecture

4. Operational Workflow
    - Construction of neural networks using foundational components
    - Selection and application of appropriate online learning algorithms
    - Model compilation generating eligibility trace computation graphs
    - Concurrent updates of model states and eligibility traces through forward propagation

BrainTrace's distinctive architecture encapsulates complex online learning algorithms behind concise interfaces while providing flexible customization mechanisms. This design philosophy achieves a balance between:
- High performance and usability
- Seamless integration with the existing BrainState ecosystem
- Intuitive and efficient construction of online learning neural networks

Through this architectural approach, BrainTrace transforms the development and training of online learning neural networks into an intuitive and efficient process, providing a powerful toolkit for neural computation research.